# Bollinger Bands Optimization Analysis (GPU Accelerated)

Questo notebook esegue una ricerca a griglia (Grid Search) accelerata via GPU per trovare i parametri ottimali delle Bande di Bollinger su dati M1 di EURGBP 2025.

In [ ]:
import sys
import os
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

# Aggiungi il percorso corrente per importare lo script
sys.path.append(os.getcwd())
from is_oos_optimizer_gpu import run_is_oos_gpu_only

# Per visualizzare plotly nel notebook
import plotly.io as pio
pio.renderers.default = "notebook_connected"

## 1. Esecuzione Ottimizzazione GPU

In [ ]:
# Definiamo i parametri da esplorare
lookbacks = np.arange(1000, 20001, 1000)
std_mults = np.arange(1.5, 5.1, 0.1)

# Esecuzione del backtest IS/OOS
results = run_is_oos_gpu_only(lookbacks, std_mults)

## 2. Visualizzazione 3D Interattiva della Superficie di Ottimizzazione

Analizziamo la stabilità dei parametri In-Sample.

In [ ]:
total_returns = results['results'][:, :, 0]
trade_counts = results['results'][:, :, 1]
expectancy = np.where(trade_counts > 0, (total_returns / trade_counts) / 0.0001, 0)

fig = make_subplots(
    rows=1, cols=2,
    specs=[[{'type': 'surface'}, {'type': 'surface'}]],
    subplot_titles=("Total Return (IS)", "Expectancy (Pips/Trade)")
)

fig.add_trace(
    go.Surface(z=total_returns, x=results['std_mults'], y=results['lookbacks'], colorscale='Viridis'),
    row=1, col=1
)

fig.add_trace(
    go.Surface(z=expectancy, x=results['std_mults'], y=results['lookbacks'], colorscale='Cividis'),
    row=1, col=2
)

fig.update_layout(
    title='Optimization Surfaces (In-Sample)',
    width=1100, height=600,
    scene=dict(xaxis_title='Std Mult', yaxis_title='Lookback', zaxis_title='Return'),
    scene2=dict(xaxis_title='Std Mult', yaxis_title='Lookback', zaxis_title='Pips')
)

fig.show()

## 3. Equity Curve (IS + OOS)

Visualizziamo il comportamento dei migliori parametri sui dati mai visti (Out-of-Sample).

In [ ]:
plt.figure(figsize=(15, 7))
is_eq = results['is_equity'].to_pandas()
oos_eq = results['oos_equity'].to_pandas() + is_eq.iloc[-1]

plt.plot(is_eq, label='In-Sample (Optimization)', color='royalblue', linewidth=2)
plt.plot(np.arange(len(is_eq), len(is_eq)+len(oos_eq)), oos_eq, label='Out-of-Sample (Validation)', color='crimson', linewidth=2)

plt.axvline(x=len(is_eq), color='black', linestyle='--', alpha=0.5)
plt.title(f"Final Equity Curve | Best LB: {results['best_lb']}, Best Std: {results['best_sm']:.2f}", fontsize=14)
plt.xlabel("Minutes")
plt.ylabel("Net Return (including commissions)")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()